## Evaluation User Behavior

In [1]:
import networkx as nx
from networkx.algorithms import community
from neo4j import GraphDatabase
import warnings
import itertools
from datetime import datetime

warnings.filterwarnings("ignore", category=FutureWarning)

URI = "bolt://localhost:7687"

password = "12345678" # CHANGE
AUTH = ("neo4j", password)

In [2]:
def get_graph_from_neo4j(driver):
    G = nx.DiGraph()
    with driver.session() as session:
        nodes_result = session.run("""
            MATCH (n)
            RETURN id(n) AS id, labels(n) AS labels, properties(n) AS properties
        """)
        for record in nodes_result:    
            G.add_node(record["id"], labels=record["labels"], **record["properties"])

        rels_result = session.run("""
            MATCH (n)-[r]->(m)
            RETURN id(n) AS source, id(m) AS target, type(r) AS type
        """)
        for record in rels_result:
            G.add_edge(record["source"], record["target"], type=record["type"])
    return G

In [3]:
def evaluate_partition(G, communities, method_name):
    if not communities or len(communities) == 0:
        print(f"No communities found by {method_name}.")
        return

    partition = [set(c) for c in communities]

    print(f"Evaluating {method_name}")
    try:
        mod = community.modularity(G, partition)
        print(f"Modularity: {mod:.4f}")
    except Exception as e:
        print(f"Could not calculate Modularity: {e}")

    try:
        qual = community.partition_quality(G, partition)
        print(f"Partition Quality (Coverage, Performance): ({qual[0]:.4f}, {qual[1]:.4f})")
    except Exception as e:
        print(f"Could not calculate Partition Quality: {e}")
    print(f"Found {len(partition)} communities.")
    print("-" * 30)

In [ ]:
def calculate_and_print_advanced_metrics(full_graph, g_shares, g_rapid_shares):
    print("\nCalculando Métricas de Coordenação")

    # Taxa de Coordenação
    try:
        total_share_pairs = g_shares.number_of_edges()
        rapid_share_pairs = g_rapid_shares.number_of_edges()
        
        if total_share_pairs > 0:
            coordination_rate = rapid_share_pairs / total_share_pairs
            print(f"  - Coordination Rate: {coordination_rate:.4f}")
    except Exception as e:
        print(f"Error: {e}")

    # User Message Uniqueness Ratio (Average)
    # User Shared Message Rate (Average)
    try:
        shared_texts_set = set()
        all_text_nodes = {n for n, d in full_graph.nodes(data=True) if 'Texto' in d.get('labels', [])}
        
        for t_node in all_text_nodes:
            users_for_this_text = set()
            for m_node in full_graph.predecessors(t_node):
                if m_node not in full_graph.nodes or 'Mensagem' not in full_graph.nodes[m_node].get('labels', []):
                    continue
                for u_node in full_graph.predecessors(m_node):
                    if u_node in full_graph.nodes and 'User' in full_graph.nodes[u_node].get('labels', []):
                        users_for_this_text.add(u_node)
            
            if len(users_for_this_text) > 1:
                shared_texts_set.add(t_node)
                
        user_nodes = {n for n, d in full_graph.nodes(data=True) if 'User' in d.get('labels', [])}
        user_uniqueness_ratios = [] 
        user_shared_rates = []  
        
        if not user_nodes:
            raise Exception("Nenhum nó 'User' encontrado no grafo completo.")

        for u_node in user_nodes:
            messages_sent = []
            for m_node in full_graph.successors(u_node):
                if m_node in full_graph.nodes and 'Mensagem' in full_graph.nodes[m_node].get('labels', []):
                    messages_sent.append(m_node)
            
            total_messages = len(messages_sent)
            if total_messages == 0:
                continue 

            unique_texts_sent = set()
            shared_message_count = 0 

            for m_node in messages_sent:
                is_shared = False
                for t_node in full_graph.successors(m_node):
                    if t_node in full_graph.nodes and 'Texto' in full_graph.nodes[t_node].get('labels', []):
                        unique_texts_sent.add(t_node)
                        
                        if t_node in shared_texts_set:
                            is_shared = True
                
                if is_shared:
                    shared_message_count += 1
            
            unique_texts = len(unique_texts_sent)
            
            user_uniqueness_ratios.append(unique_texts / total_messages)
            user_shared_rates.append(shared_message_count / total_messages)
        
        if user_uniqueness_ratios:
            avg_user_uniqueness = sum(user_uniqueness_ratios) / len(user_uniqueness_ratios)
            print(f"  - User Message Uniqueness Ratio (Average): {avg_user_uniqueness:.4f}")
        
        if user_shared_rates:
            avg_user_shared_rate = sum(user_shared_rates) / len(user_shared_rates)
            print(f"  - User Shared Message Rate (Average): {avg_user_shared_rate:.4f}")
            
    except Exception as e:
        print(f"Error: {e}")

In [5]:
import traceback

G_shares = nx.Graph()
G_viral = nx.Graph()
G_misinfo = nx.Graph()

G_rapid_shares = nx.Graph()

try:
    driver = GraphDatabase.driver(URI, auth=AUTH)

    print("Connecting to Neo4j and building NetworkX graph...")

    networkx_graph = get_graph_from_neo4j(driver)

    print("Graph imported into NetworkX")

    user_nodes = {n for n, d in networkx_graph.nodes(data=True) if 'User' in d.get('labels', [])}

    for u_node in user_nodes:
        G_shares.add_node(u_node, **networkx_graph.nodes[u_node])
        G_viral.add_node(u_node, **networkx_graph.nodes[u_node])
        G_misinfo.add_node(u_node, **networkx_graph.nodes[u_node])
        G_rapid_shares.add_node(u_node, **networkx_graph.nodes[u_node])

    for u, v, data in networkx_graph.edges(data=True):
        if u in user_nodes and v in user_nodes:
            edge_type = data.get('type')
            if edge_type == 'SHARES':
                G_shares.add_edge(u, v, **data)
            elif edge_type == 'VIRAL_SHARES':
                G_viral.add_edge(u, v, **data)
            elif edge_type == 'SHARES_MISINFORMATION':
                G_misinfo.add_edge(u, v, **data)
            elif edge_type == "RAPID_SHARE":
                G_rapid_shares.add_edge(u, v, **data)

    G_shares.remove_nodes_from(list(nx.isolates(G_shares)))
    G_viral.remove_nodes_from(list(nx.isolates(G_viral)))
    G_misinfo.remove_nodes_from(list(nx.isolates(G_misinfo)))
    G_rapid_shares.remove_nodes_from(list(nx.isolates(G_rapid_shares)))

    graphs_to_analyze = {
        "SHARES": G_shares,
        "VIRAL_SHARES": G_viral,
        "SHARES_MISINFORMATION": G_misinfo,
        "RAPID_SHARE": G_rapid_shares
    }

    for name, G in graphs_to_analyze.items():
        print(f"\nAnalisando Grafo: '{name}'")

        try:
            communities = nx.community.louvain_communities(G, weight='weight')
            evaluate_partition(G, communities, "Louvain")
        
        except Exception as e:
            print(f"Erro ao processar comunidades para '{name}': {e}")

    calculate_and_print_advanced_metrics(networkx_graph, G_shares, G_rapid_shares)

except Exception:
    traceback.print_exc()
finally:
    if 'driver' in locals() and driver:
        driver.close()
        print("Connection closed.")

Connecting to Neo4j and building NetworkX graph...


Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. id is deprecated. It is replaced by elementId or consider using an application-generated id.', position=<SummaryInputPosition line=3, column=20, offset=42>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 42, 'line': 3, 'column': 20}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: '\n            MATCH (n)\n            RETURN id(n) AS id, labels(n) AS labels, properties(n) AS properties\n        '
Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. id is deprecated. It is replaced by elementId or consider

Graph imported into NetworkX

Analisando Grafo: 'SHARES'
Evaluating Louvain
Modularity: 0.4663
Partition Quality (Coverage, Performance): (0.5759, 0.9268)
Found 142 communities.
------------------------------

Analisando Grafo: 'VIRAL_SHARES'
Evaluating Louvain
Modularity: 0.0307
Partition Quality (Coverage, Performance): (0.4286, 0.5604)
Found 3 communities.
------------------------------

Analisando Grafo: 'SHARES_MISINFORMATION'
Evaluating Louvain
Modularity: 0.6488
Partition Quality (Coverage, Performance): (0.7658, 0.9245)
Found 45 communities.
------------------------------

Analisando Grafo: 'RAPID_SHARE'
Evaluating Louvain
Modularity: 0.9744
Partition Quality (Coverage, Performance): (1.0000, 0.9949)
Found 62 communities.
------------------------------

Calculando Métricas de Coordenação
  - Coordination Rate: 0.0013
  - User Message Uniqueness Ratio (Average): 0.9360
  - User Shared Message Rate (Average): 0.1850
Connection closed.
